In [ ]:
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
from tqdm import tqdm

VGG NET

In [ ]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [ ]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(nn.Linear(512*7*7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096,4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg:
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1, bias=False),
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]
        return nn.Sequential(*layers)

In [ ]:
model = VGG(cfgs['E'], batch_norm=False)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,32,32), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [2, 64, 32, 32]           1,792
│    └─ReLU: 2-2                         [2, 64, 32, 32]           --
│    └─Conv2d: 2-3                       [2, 64, 32, 32]           36,928
│    └─ReLU: 2-4                         [2, 64, 32, 32]           --
│    └─MaxPool2d: 2-5                    [2, 64, 16, 16]           --
│    └─Conv2d: 2-6                       [2, 128, 16, 16]          73,856
│    └─ReLU: 2-7                         [2, 128, 16, 16]          --
│    └─Conv2d: 2-8                       [2, 128, 16, 16]          147,584
│    └─ReLU: 2-9                         [2, 128, 16, 16]          --
│    └─MaxPool2d: 2-10                   [2, 128, 8, 8]            --
│    └─Conv2d: 2-11                      [2, 256, 8, 8]            29

Train

In [ ]:
batch_size = 128
learning_rate = 0.0001
num_epoch = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG(cfgs['D'], batch_norm = False).to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.ToTensor()
cifar10_train = datasets.CIFAR10(root="../Data/", train=True, transform=transform, target_transform=None, download=True)
cifar10_test = datasets.CIFAR10(root="../Data/", train=False, transform=transform, target_transform=None, download=True)
train_DL = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_DL = torch.utils.data.DataLoader(cifar10_test, batch_size=batch_size, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:13<00:00, 13035607.21it/s]


Extracting ../Data/cifar-10-python.tar.gz to ../Data/
Files already downloaded and verified


In [ ]:
NoT = len(train_DL)

for i in range(num_epoch):
    rloss = 0
    for image,label in tqdm(train_DL):
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        rloss += loss.item()

    loss_e = rloss/NoT
    print(f"Epoch: {i+1}, train loss: {round(loss_e,3)}")
    print("-"*20)

  0%|          | 0/391 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 391/391 [00:44<00:00,  8.74it/s]


Epoch: 1, train loss: 2.097
--------------------


100%|██████████| 391/391 [00:44<00:00,  8.75it/s]


Epoch: 2, train loss: 1.38
--------------------


100%|██████████| 391/391 [00:44<00:00,  8.80it/s]


Epoch: 3, train loss: 1.104
--------------------


100%|██████████| 391/391 [00:44<00:00,  8.79it/s]


Epoch: 4, train loss: 0.925
--------------------


100%|██████████| 391/391 [00:44<00:00,  8.79it/s]

Epoch: 5, train loss: 0.783
--------------------


Test

In [ ]:
model.eval()
with torch.no_grad():
    rcorrect = 0
    for image, label in test_DL:
        x = image.to(device)
        y = label.to(device)

        output = model.forward(x)

        pred = output.argmax(dim=1)
        corrects_b = torch.sum(pred == y).item()
        rcorrect += corrects_b
    accuracy_e = rcorrect/len(test_DL.dataset)*100
print(f"Test accuracy: {rcorrect}/{len(test_DL.dataset)} ({round(accuracy_e,1)} %)")

Test accuracy: 6775/10000 (67.8 %)
